# Master's Thesis: Analysis of Carbon Transition Risk #
Arthur Enders, RWTH Aachen University

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pickle

# define data path
with open('data_path.txt') as f:
    lines = f.read().splitlines()
data_path = lines[0]

# start year = 2007
# end year = 2021

In [17]:
# Read IDs
# ID Data: RIC, ISIN, Name, Country

EU_IDs = pd.read_csv(data_path+"EU IDs.csv",sep=';')
US_IDs = pd.read_csv(data_path+"USA IDs.csv", sep=';')
US_OTC_IDs = pd.read_csv(data_path+"USA_OTC IDs.csv", sep=';')

# merge
ids = pd.concat([US_IDs,US_OTC_IDs,EU_IDs])
ids = ids.drop_duplicates()
# LILM.O duplicate, former SPAC merged with German country -> drop USA value
ids = ids.drop(ids[(ids.RIC == "LILM.O") & (ids["Country of Issuer"] == "USA")].index)

# rename columns
ids = ids.rename(columns={"Company Common Name": "Name", "TRBC Economic Sector Name": "Sector","Country of Issuer":"Country"})

#,"Institutions, Associations & Organizations",ids["Sector"])

ids = ids.reset_index(drop=True)

ids["Sector"] = np.where(ids["Sector"] == "Institutions, Associations, Organizations","Institutions, Associations & Organizations",ids["Sector"])

In [18]:
# Third Party Factor Data and Risk Free Rates
# Fama French Factors from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
# EU: Fama/French European 3 Factors
EU_3F = pd.read_csv(data_path+"Europe_3_Factors.csv",sep=',')
EU_3F["Date"] = pd.to_datetime(EU_3F['Date'], format='%Y%m')

EU_WML = pd.read_csv(data_path+"Europe_MOM_Factor.csv",sep=',')
EU_WML["Date"] = pd.to_datetime(EU_WML['Date'], format='%Y%m')

EU_4F = EU_3F.merge(EU_WML,"left",left_on="Date",right_on="Date")

# US Fama/French 3 Factors
US_3F = pd.read_csv(data_path+"F-F_Research_Data_Factors.csv",sep=',')
US_3F["Date"] = pd.to_datetime(US_3F['Date'], format='%Y%m')

# US Momentum Factor
US_WML = pd.read_csv(data_path+"F-F_Momentum_Factor.csv",sep=',')
US_WML["Date"] = pd.to_datetime(US_WML['Date'], format='%Y%m')

US_4F = US_3F.merge(US_WML,"left",left_on="Date",right_on="Date")

# Global Carbon Risk Factors from Görgen et.al.: https://www.uni-augsburg.de/de/fakultaet/wiwi/prof/bwl/wilkens/sustainable-finance/downloads/
#BMG_F = pd.read_csv(data_path+"carbon_risk_factors.csv",sep=',')
#BMG_F["Date"] = pd.to_datetime(BMG_F['Date'], format='%Y-%m-%d')

# change monthly dates to last day of month instead of first
import datetime
def last_day_of_month(any_day):
    # this will never fail
    # get close to the end of the month for any day, and add 4 days 'over'
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    # subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return next_month - datetime.timedelta(days=next_month.day)

US_4F["Date"] = US_4F["Date"].apply(last_day_of_month)
EU_4F["Date"] = EU_4F["Date"].apply(last_day_of_month)
#BMG_F["Date"] = BMG_F["Date"].apply(last_day_of_month)

In [19]:
# read Returns
# Return Data: Monthly Data (Frq=M)
# Example 30/09/2021: ((Adjusted Close_August31/Adjusted Close_September30)-1)*100

EU_returns = pd.read_csv(data_path+"EU returns.csv",sep=",")
US_returns = pd.read_csv(data_path+"USA returns.csv",sep=",")
US_OTC_returns = pd.read_csv(data_path+"USA_OTC returns.csv",sep=",")

# merge
returns = pd.concat([US_returns,US_OTC_returns,EU_returns], axis=1)
returns["RiskfreeEU"] = EU_4F["RF"]
returns["RiskfreeUS"] = US_4F["RF"]
# remove duplicates and NaN values
returns = returns.loc[:,~returns.columns.duplicated()]
returns = returns.fillna(0)
returns = returns.replace("Unable to collect data for the field 'TR.TOTALRETURN' and some specific identifier(s).",0)
# date string to datetime
returns['Date'] = pd.to_datetime(returns['Date'], format='%d/%m/%Y')


In [20]:
# Read Key Data
# Currency USD
# MarketCap/Revenue/SharesOuts/TotalEquity: Scale 6 (Millions)
# AnalyticCO2: normalized CO2 emissions by Reuters: Total CO2/Total Revenue

EU_KeyData = pd.read_csv(data_path+"EU KeyData.csv",sep=",")
US_KeyData = pd.read_csv(data_path+"USA KeyData.csv", sep=',')
US_OTC_KeyData = pd.read_csv(data_path+"USA_OTC KeyData.csv", sep=',')

# merge
keydata = [US_KeyData,US_OTC_KeyData,EU_KeyData]
keydata = pd.concat(keydata)
# remove rows without RIC or Date
keydata.dropna(subset = ["RIC"], inplace=True)
keydata.dropna(subset = ["Date"], inplace=True)
# NaN to 0
keydata = keydata.fillna(0)

# date string to datetime
keydata['Date'] = pd.to_datetime(keydata['Date'], format='%d/%m/%Y')

# remove duplicates
keydata = keydata.drop_duplicates()
keydata = keydata.drop_duplicates(subset=['RIC',"Date"], keep="first")


# remove "Unable ..." strings
for c in keydata.columns:
    keydata = keydata.replace("Unable to collect data for the field '"+c+"' and some specific identifier(s).",0)

keydata = keydata.reset_index(drop=True)

# rename columns
keydata = keydata.rename(columns={"ZAV(TR.CO2EmissionTotal)": "TotalCO2", "ZAV(TR.CompanyMarketCap(Scale=6))": "MarketCap",
"ZAV(TR.CO2IndirectScope3)":"CO2Scope3","ZAV(TR.AnalyticCO2)":"AnalyticCO2","ZAV(TR.Revenue(Scale=6))":"Revenue",
"ZAV(TR.TRESGScore)":"ESG","ZAV(TR.BookValuePerShare)":"BVpershare","ZAV(TR.CompanySharesOutstanding(Scale=6))":"NumShares",
        "ZAV(TR.TotalEquity(Scale=6))":"BookValue"})

# convert values to floats
keydata["BookValue"] = pd.to_numeric(keydata["BookValue"], downcast="float")
keydata["MarketCap"] = pd.to_numeric(keydata["MarketCap"], downcast="float")
keydata["TotalCO2"] = pd.to_numeric(keydata["TotalCO2"], downcast="float")
keydata["AnalyticCO2"] = pd.to_numeric(keydata["AnalyticCO2"], downcast="float")


# Add Book/Market ratio
keydata["BtoM"] = keydata["BookValue"]/keydata["MarketCap"]
keydata.replace([np.inf, -np.inf], 0, inplace=True)
keydata = keydata.fillna(0)


In [21]:
# Monthly Market Cap Data for Portfolio Formulation
EU_MonMC = pd.read_csv(data_path+"EU MonMC.csv",sep=",")
US_MonMC = pd.read_csv(data_path+"USA MonMC.csv", sep=',')
US_OTC_MonMC = pd.read_csv(data_path+"USA_OTC MonMC.csv", sep=',')

# merge
mcdata = [US_MonMC,US_OTC_MonMC,EU_MonMC]
mcdata = pd.concat(mcdata, axis=1)

# remove duplicates and NaN values
mcdata = mcdata.loc[:,~mcdata.columns.duplicated()]
mcdata = mcdata.fillna(0)
mcdata = mcdata.replace("Unable to collect data for the field 'ZAV(TR.CompanyMarketCap(Scale=6))' and some specific identifier(s).",0)
# date string to datetime
mcdata = mcdata[mcdata.Date != 0]
mcdata['Date'] = pd.to_datetime(mcdata['Date'], format='%d/%m/%Y')

In [23]:
# Save Data for use in other files
f = open(data_path+'clean_data.pckl', 'wb')
pickle.dump([mcdata, keydata, ids,returns, EU_4F, US_4F], f)
f.close()